<a href="https://colab.research.google.com/github/t1seo/42Curcus/blob/master/04_Machine_Learning/%ED%98%BC%EC%9E%90%20%EA%B3%B5%EB%B6%80%ED%95%98%EB%8A%94%20%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%20%EB%94%A5%EB%9F%AC%EB%8B%9D/07%20%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84%20%EC%8B%9C%EC%9E%91%ED%95%A9%EB%8B%88%EB%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07-1 인공 신경망

## 패션 MNIST

In [ ]:
from tensorflow import keras

# 데이타 다운로드
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

In [ ]:
print(train_input.shape, train_target.shape)

In [ ]:
# 몇 개의 샘플을 그림으로 출력
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 10, figsize=(10 ,10))
for i in range(10):
    axs[i].imshow(train_input[i], cmap='gray_r')
    axs[i].axis('off')
plt.show()

In [ ]:
print(test_input.shape, test_target.shape)

In [ ]:
# 10개의 샘플 타깃값을 리스트로 만든 후 출력
print([train_target[i] for i in range(10)])

In [ ]:
# 샘플 개수 확인
import numpy as np
print(np.unique(train_target, return_counts=True))

## 로지스틱 회귀로 패션 아이템 분류하기

In [ ]:
# 각 샘플을 1차원 배열로 만들기
train_scaled = train_input / 255.0 # 각 픽셀 값을 0~1 사이의 값으로 정규화
train_scaled = train_scaled.reshape(-1, 28*28)

In [ ]:
print(train_scaled.shape)

In [ ]:
# 교차 검증
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier

sc = SGDClassifier(loss='log', max_iter=5, random_state=42)
scores = cross_validate(sc, train_scaled, train_target, n_jobs=-1)
print(np.mean(scores['test_score']))

## 인공 신경망

- 인공 신경망
- 출력층
- 뉴런
- 입력층

### 텐서플로와 케라스

In [ ]:
# 텐서플로
import tensorflow as tf

In [ ]:
# 케라스
from tensorflow import keras

## 인공 신경망으로 모델 만들기

In [ ]:
# 검증 세트 나누기
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 훈련세트와 검증 세트의 크기
print(train_scaled.shape, train_target.shape)

In [ ]:
print(val_scaled.shape, val_target.shape)

In [ ]:
# 밀집층 만들기
dense = keras.layers.Dense(10, activation='softmax', input_shape=(784,)) # 뉴런 개수, 뉴런의 출력에 적용할 함수, 입력의 크기

- 10개의 패션 아이템을 분류하기 때문에 뉴런 개수 10개
- 뉴런에서 출력되는 값을 확률로 바꾸기 위해 소프트맥스 함수 상요
- 10개의 뉴런이 각각 몇 개의 입력을 받는지 튜플로 지정

In [ ]:
# 신경망 모델 만들기
model = keras.Sequential(dense) # 밀집층의 객체를 전달

## 인공 신경망으로 패션 아이템 분류하기

In [ ]:
# 케라스 모델 훈련하기 전에 설정
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy') # 손실 함수 종류, 측정값

In [ ]:
print(train_target[:10])

텐서플로에서는 정수로 된 타깃값을 원-핫 인코딩으로 바꾸지 않고 그냥 사용할 수 있다. 정수로된 타깃값을 사용해 크로스 엔트로피 손실을 계싼하는 것이 바로 `sparse_categorical_crossentropy`이다.

In [ ]:
# 훈련
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
# 모델 성능 평가
model.evaluate(val_scaled, val_target)

# 07-2 심층 신경망

## 2개의 층

In [ ]:
# MNIST 데이터셋 불러오기
from tensorflow import keras

(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

In [ ]:
# 훈련 세트와 검증 세트로 나누기
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 은닉층과 출력층 추가
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

## 심층 신경망 만들기

In [ ]:
# 심층 신경망(DNN) 만들기
model = keras.Sequential([dense1, dense2])

In [ ]:
# 층에 대한 정보 출력
model.summary()

## 층을 추가하는 다른 방법

In [ ]:
# dense를 직접 입력
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [ ]:
model.summary()

In [ ]:
# Sequential 클래스의 객체에 add() 메서드로 층을 추가
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

In [ ]:
model.summary()

In [ ]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

## 렐루 함수

In [ ]:
# ReLU 함수 사용
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 1차원으로 펼치기
model.add(keras.layers.Dense(10, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# 훈련데이터를 다시 준비
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 모델을 컴파일하고 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
# 검증 세트의 성능 확인
model.evaluate(val_scaled, val_target)

## 옵티마이저
케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하는데 이들을 옵티마이저optimizer라 부른다.

In [ ]:
# 신경망 만들기
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
# adam 옵티마이저로 훈련
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

# 07-3 신경망 모델 훈련

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# MNIST 데이터셋 불러오기
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

# 훈련 세트와 검증 세트로 나누기

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 모델을 만드는 함수
def model_fn(a_layer=None):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    model.add(keras.layers.Dense(100, activation='relu'))
    if a_layer:
        model.add(a_layer)
    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

In [ ]:
model = model_fn()
model.summary()

In [ ]:
# fit() 메서드의 결과를 history 변수에 담는다
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=5, verbose=0)

In [ ]:
print(history.history.keys())

In [ ]:
# 손실을 그래프로 그리기
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# 정확도를 그래프로
# 손실을 그래프로 그리기
plt.plot(history.history['accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
# 에포크 횟수를 20으로 증가
model = model_fn()
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=20, verbose=0)
plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

## 검증 손실
에포크에 따른 과대적합과 과소적합을 파악하려면 훈련 세트에 대한 점수뿐만 아니라 검증 세트에 대한 점수도 필요하다.

In [ ]:
# 에포크마다 검증 손실 계산
model = model_fn()
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# Adam 옵티마이저 사용
model = model_fn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

Adam은 적응적 학습률을 사용하기 떄문에 에포크가 진행되면서 학습률의 크기를 조정할 수 있다.

## 드롭아웃
드롭아웃(dropout)은 제프리 힌턴이 소개한 개념으로 훈련 괒어에서 층에 있는 일부 뉴런을 랜덤하게 꺼서(뉴런의 출력을 0으로 만들어)과대 적합을 막는 방버이다.

이전 층의 일부 뉴런이 랜덤하게 꺼짐녀 특정 뉴런에 과대하게 의존하는 것을 줄일 수 있고 모든 입력에 대해 주의를 기울여야 하기 때문에 드롭아웃 방법은 과대적합을 막는다.

In [ ]:
# model_fn() 함수에 드롭아웃 객체를 전달하여 층을 추가
model = model_fn(keras.layers.Dropout(0.3))
model.summary()

In [ ]:
# 훈련 손실과 검증 손실의 그래프 비교
model = model_fn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

## 모델 저장과 복원


In [ ]:
model = model_fn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=10, verbose=0,
                    validation_data=(val_scaled, val_target))

In [ ]:
# 훈련된 모델의 파라미터 저장
model.save_weights('model-weights.h5')

In [ ]:
# 모델 구조와 모델 파라미터를 함께 저장
model.save('model-whole.h5')

In [ ]:
# 두 파일 확인
!ls -al *.h5

In [ ]:
# 훈련하지 않은 새로운 모델을 만들고 모델 파라미터 읽어서 사용
model = model_fn(keras.layers.Dropout(0.3))
model.load_weights('model-weights.h5')

In [ ]:
# 모델의 정확도 확인 위해 10개 확률 중 가장 큰 인덱스를 골라 타깃 레이블과 비교
import numpy as np

val_labels = np.argmax(model.predict(val_scaled), axis=-1) # axis=-1은 배열의 마지막 차원을 따라 최댓값 고른다
print(np.mean(val_labels == val_target))

In [ ]:
# 모델 전체를 파일에서 읽은 당므 검증 세트의 정확도 출력
model = keras.models.load_model('model-whole.h5')
model.evaluate(val_scaled, val_target)

## 콜백
콜백(callback)은 훈련 과정 중간에 어떤 작업을 수행할 수 있게 하는 객체로 `keras.callbacks` 패키지 아래에 있는 클래스들이다. `fit()` 메서드의 callbacks 매개변수에 리스트로 전달하여 사용한다.

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5') # 모델이 훈련한 후에 최상의 점수를 낸 모델이 저장된다a
model.fit(train_scaled, train_target, epochs=20, verbose=0,
          validation_data=(val_scaled, val_target),
          callbacks=[checkpoint_cb])

In [ ]:
model = keras.models.load_model('best-model.h5')
model.evaluate(val_scaled, val_target)

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5') # 모델이 훈련한 후에 최상의 점수를 낸 모델이 저장된다a
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                 restore_best_weights=True) # 조기 종료
history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
          validation_data=(val_scaled, val_target),
          callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
# 몇 번째 에포크에서 훈련이 중지되었는지 확인
print(early_stopping_cb.stopped_epoch)

In [ ]:
# 훈련 손실과 검증 손실을 출력해서 확인
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# 조기 종료로 얻은 모델을 사용해 검증 세트에 대한 성능 확인
model.evaluate(val_scaled, val_target)